In [15]:
contactSheets = [x.replace(os.sep,'/') for x in glob(path)]
cs_splits = [x.split('/') for x in contactSheets]
cs_splitspds = pd.Series([x.split('/') for x in contactSheets])

df = pd.DataFrame(cs_splits)
# df['name'] = df[4]
# df

df = pd.DataFrame([x.replace(os.sep,'/').split('/') for x in glob(path)])
df
# unique in column
print(df[4].unique())

# unique+count in column
print(df[4].value_counts())



['CamilaMazurek' 'ChrisHemsworth' 'JasonMomoa' 'KamilaTen' 'LucyMae'
 'ManuelTucker' 'MonstaXJooheon' 'PaulSamatar' 'PhilipAn' 'RonaldNelson'
 'AmandaMoore' 'AliceRivera' 'WandaEdwards' 'VeronicaYoung' 'LouisPrice'
 'SimonYuen']
SimonYuen         26
CamilaMazurek     18
RonaldNelson      16
ChrisHemsworth    14
KamilaTen         14
LucyMae           14
ManuelTucker      14
MonstaXJooheon    14
PaulSamatar       14
PhilipAn          14
VeronicaYoung     14
LouisPrice        14
JasonMomoa        13
AmandaMoore       12
WandaEdwards      11
AliceRivera        2
Name: 4, dtype: int64


In [1]:
from glob import glob
import pandas as pd

path = "G:/Shared drives/TriplegangersGroom_ext/Groom_INTERNAL/*/maya/base_delta/images/*/contactSheet.png"

df = pd.DataFrame([x.replace(os.sep,'/').split('/') for x in glob(path)])
df
# unique in column
print(df[4].unique())

# unique+count in column
print(df[4].value_counts())

['CamilaMazurek' 'ChrisHemsworth' 'JasonMomoa' 'KamilaTen' 'LucyMae'
 'ManuelTucker' 'MonstaXJooheon' 'PaulSamatar' 'PhilipAn' 'RonaldNelson'
 'AmandaMoore' 'AliceRivera' 'WandaEdwards' 'VeronicaYoung' 'LouisPrice'
 'SimonYuen']
SimonYuen         26
CamilaMazurek     18
RonaldNelson      16
ChrisHemsworth    14
KamilaTen         14
LucyMae           14
ManuelTucker      14
MonstaXJooheon    14
PaulSamatar       14
PhilipAn          14
VeronicaYoung     14
LouisPrice        14
JasonMomoa        13
AmandaMoore       12
WandaEdwards      11
AliceRivera        2
Name: 4, dtype: int64


In [88]:
import os
import re
import numpy as np
from glob import glob
import tomli
from itertools import product, permutations


# pathing
sourcePath = "G:/Shared drives/TriplegangersGroom_ext/Groom_INTERNAL/"
configPath = "0000_base_delta/template/"
xgd_configFile = "delta_c000.toml"
xgPath = "/maya/base/scenes/base__head_coll.xgen"
maPath = "/maya/base/scenes/base.ma"
deltaPath = "maya/base_delta/scenes/deltaGen/"


def get_grooms(path:str=sourcePath, name:str="**"):
    """ get grooms from source path.  default name = '**' """
    print(f'search path: {path}, name filters: {name}')
    exclude = ["turntableQC", "0000_base_delta", "image_DB"]
    groomList = [x.replace(os.sep, '/').split('/')[-1] for x in glob(path+name)]
    groomList = [x for x in groomList if x not in exclude]
    return groomList

def clean_deltas(groomName:str = '**'):
    """removes all ../scenes/delta/ folders"""
    import shutil
    for groom in get_grooms(sourcePath, groomName):
        print(groom)
        # shutil.rmtree(deltaOutPath(groom))
    print("clean complete")

clean_deltas(groomName='AmandaMoore')

search path: G:/Shared drives/TriplegangersGroom_ext/Groom_INTERNAL/, name filters: AmandaMoore
AmandaMoore
clean complete


In [179]:
def prune(desc:list="", discard:list=["eye", "baby"]) -> list:
    """ with input list, remove discards """
    descp = [x for x in desc if not any(word in x.lower() for word in discard)]
    discards = [x for x in desc if any(word in x.lower() for word in discard)]
    # print("retained:", descp), print("discarded:", discards)
    return descp


def get_grooms(path:str=sourcePath, name:str="**"):
    """ get grooms from source path.  default name = '**' """
    print(f'search path: {path}, name filters: {name}')
    exclude = ["turntableQC", "0000_base_delta", "image_DB"]
    groomList = [x.replace(os.sep, '/').split('/')[-1] for x in glob(path+name)]
    groomList = [x for x in groomList if x not in exclude]
    return groomList


def get_desc(groomName):
    '''
    returns descriptions from .xgen file
    '''
    xgenPath = f'{sourcePath}{groomName}{xgPath}'
    desc_names=[]
    xg_file =  open(xgenPath, 'r')
    lines = xg_file.readlines()
    xg_file.close()
    for i in range(len(lines)):
        ck = lines[i]
        if ck.startswith("Description"):
            desc_names.append(lines[i+1].rpartition("\t")[2].rstrip("\n"))
    return desc_names

def deltaOutPath(groomName):
    """creates delta output path if its doesnt exist"""
    outputPath = "{0}{1}/{2}".format(sourcePath, groomName, deltaPath)
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
        print("created dir:", outputPath)
    return outputPath

def __load_config():
    """open config file with read + binary"""
    with open(sourcePath+configPath+xgd_configFile, 'rb') as f:
        config = tomli.load(f)
    return config

def file_reset(filename:str=""):
    with open(filename, 'w') as f:
        f.write("")
        f.close()

In [276]:
def inputTest(groomName:str, collection:str, modifier:str, **kwargs):
    """
    take groomName, collection, modifier, and kwargs as input; XGen xgd file as output

    available modifiers:
    dh_coil: [count, radius], mask
    dh_cutClamp: [cutLength], mask
    dh_cutPercent: [percent], mask
    dh_noiseGen: [frequency, magnitude, correlation], mask
    dh_exp_gScale: [gScale]
    dh_wind: [direction, stiffness, constStrength, ]
    """

    #  get descriptions from groom
    palettes = prune(get_desc(groomName))

    x = [x for x in kwargs] # kwargs list
    p = product(*[kwargs.get(x) for x in kwargs]) # cartesian product of kwarg values

    for a in p:
        xpzip = list(zip(x, a))
        name_list = []
        for w in xpzip:
            name_list.extend([w[0][:1],w[1]])
        filename  = deltaOutPath(groomName)+f'{groomName}__{collection}__{modifier}__{"_".join([str(x) for x in name_list])}.xgd'
        file_reset(filename)
        for desc in palettes:
            with open(filename, 'a') as f:
                workfile = __load_config()["modifiers"][f'{modifier}'][0]
                workfile = workfile.replace("<xgenDescName>", desc)
                for w in xpzip:
                    workfile = workfile.replace(f'<{w[0]}>', str(w[1]))
                    # print(f'replacing <{w[0]}> with {str(w[1])}')
                f.write(workfile)


inputTest('AliceRivera', 'head', 'dh_coil', count=[1, 2], frequency=[1, 12], magnitude=[2, 23])
inputTest('AliceRivera', 'head', 'dh_cutClamp', cut_length=[.1, .5, 2, 5, 10])
inputTest('AliceRivera', 'head', 'dh_exp_gScale', gScale=[.1, .3, .5, .7, 1.3, 2.1, 3.4])
inputTest('AliceRivera', 'head', 'dh_noise', frequency=[1,2,13], magnitude=[3,56,6])

In [196]:
def testInput(**kwargs):
    x = [x for x in kwargs] # kwargs list
    p = product(*[kwargs.get(x) for x in kwargs]) # cartesian product of kwarg values

    for a in p:
        xpzip = list(zip(x, a))
        name_list = []
        for w in xpzip:
            name_list.extend([w[0],w[1]])
        filename  = f'{"_".join([str(x) for x in name_list])}.ext'
        print(filename)
testInput(a=[1,2], b=[3,4,5,6], c=[7])

a_1_b_3_c_7.ext
a_1_b_4_c_7.ext
a_1_b_5_c_7.ext
a_1_b_6_c_7.ext
a_2_b_3_c_7.ext
a_2_b_4_c_7.ext
a_2_b_5_c_7.ext
a_2_b_6_c_7.ext
